# OpenAI Thinking Parameters with GPT-5.2

This notebook demonstrates OpenAI's GPT-5.2 reasoning capabilities using the **Responses API**.

## What Are Reasoning Models?

Reasoning models emit **hidden reasoning tokens** before generating their final answer. This allows them to:
- Break down complex problems into steps
- Explore multiple approaches before committing to an answer
- Verify their work and catch mistakes
- Handle multi-step tasks more reliably

Think of it like showing your work in math class - the model "thinks through" the problem before answering.

## GPT-5.2 Overview

| Feature | Value |
|---------|-------|
| Context Window | 400K tokens |
| Input Cost | $1.75 per M tokens |
| Cached Input | $0.175 per M tokens |
| Output Cost | $14.00 per M tokens |

## Table of Contents

1. [Setup](#setup)
2. [Responses API Basics](#responses-api)
3. [Reasoning Effort Parameter](#reasoning-effort)
4. [Verbosity Control](#verbosity)
5. [Tool Use with Reasoning](#tools)
6. [Structured Outputs](#structured)
7. [Best Practices](#best-practices)
8. [Practical Examples](#examples)
9. [Summary](#summary)

<a id='setup'></a>
## 1. Setup

First, let's set up our environment and initialize the OpenAI client.

In this section you will:
- Load your API key from `.env` (or be prompted once)
- Create the `OpenAI()` client
- Confirm setup with a short status message

Expected output: a single line confirming the client is initialized.

In [2]:
import os
import getpass
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
load_dotenv()

# Set API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

client = OpenAI()
print("OpenAI client initialized (ready for API calls)")

OpenAI client initialized (ready for API calls)


<a id='responses-api'></a>
## 2. Responses API Basics

GPT-5.2 uses the **Responses API** (`client.responses.create`) instead of the older Chat Completions API. This API is specifically designed for reasoning models.

### Key Differences from Chat Completions

| Feature | Chat Completions | Responses API |
|---------|-----------------|---------------|
| Method | `chat.completions.create()` | `responses.create()` |
| Messages | `messages=[...]` | `input=[...]` |
| Roles | system, user, assistant | developer, user |
| Output | `response.choices[0].message.content` | `response.output_text` |
| Reasoning | Not available | `reasoning={"effort": "..."}` |

Goal: see the smallest possible request/response loop with the Responses API.
Watch for:
- `response.output_text` (the model's answer)
- `response.usage.total_tokens` (cost/latency proxy)

In [3]:
# Basic usage pattern with the Responses API
print("Responses API basic example")
print("-" * 60)

response = client.responses.create(
    model="gpt-5.2",
    input=[
        {
            "role": "developer",
            "content": "You are a helpful assistant that provides clear, concise answers."
        },
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ]
)

print(f"Answer: {response.output_text}")
print(f"Total tokens: {response.usage.total_tokens}")

Responses API basic example
------------------------------------------------------------
Answer: Paris.
Total tokens: 35


<a id='reasoning-effort'></a>
## 3. Reasoning Effort Parameter

GPT-5.2 supports five reasoning effort levels:

| Level | Use When | Speed | Cost | Quality |
|-------|----------|-------|------|----------|
| **none** | No reasoning needed, fastest response | Fastest | Lowest | Basic |
| **low** | Simple tasks with minimal reasoning | Fast | Low | Good |
| **medium** | Balanced default for most workflows | Moderate | Moderate | Great |
| **high** | Complex multi-step tasks, critical accuracy | Slower | Higher | Best |
| **xhigh** | Extremely complex problems, maximum accuracy | Slowest | Highest | Maximum |

**Key Insight**: Higher effort = more reasoning tokens = better accuracy but higher latency/cost

We will run the same model with different `reasoning.effort` levels.
As you go, compare output quality and the reported reasoning token counts.

In [ ]:
# Example: No reasoning - Quick classification
print("Reasoning effort = none (sentiment classification)")
print("-" * 60)

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "none"},
    input=[
        {
            "role": "developer",
            "content": "Classify the sentiment as: positive, neutral, or negative. Return only one word."
        },
        {
            "role": "user",
            "content": "The new update completely broke my workflow. Very disappointed."
        }
    ]
)

print(f"Predicted sentiment: {response.output_text}")
print(f"Total tokens: {response.usage.total_tokens}")

Reasoning effort = none (sentiment classification)
------------------------------------------------------------
Predicted sentiment: negative
Total tokens: 44


In [5]:
# Example: Medium reasoning - Code generation
print("Reasoning effort = medium (email validator)")
print("-" * 60)

prompt = """
Write a Python function that validates an email address.
Requirements:
- Check for @ symbol
- Verify domain has at least one dot
- Ensure no spaces
- Return True/False

Include a docstring and 2-3 test cases.
"""

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}]
)

print("Generated function and tests:")
print(response.output_text)
print(f"\n{'='*60}")
print(f"Reasoning tokens: {response.usage.output_tokens_details.reasoning_tokens}")

Reasoning effort = medium (email validator)
------------------------------------------------------------
Generated function and tests:
```python
def is_valid_email(email: str) -> bool:
    """
    Validate a basic email address format.

    Rules:
    - Must contain exactly one '@'
    - Must contain no spaces
    - Domain part (after '@') must contain at least one '.'

    This is a simple validator and does not fully implement RFC email rules.
    """
    if not isinstance(email, str):
        return False

    if " " in email:
        return False

    if email.count("@") != 1:
        return False

    local, domain = email.split("@")
    if not local or not domain:
        return False

    if "." not in domain:
        return False

    return True


# Test cases
print(is_valid_email("user@example.com"))   # True
print(is_valid_email("userexample.com"))    # False (missing '@')
print(is_valid_email("user@domain"))        # False (domain missing '.')
```

Reasoning tokens: 0


In [6]:
# Example: High reasoning - Complex algorithmic problem
print("Reasoning effort = high (longest palindromic substring)")
print("-" * 60)

problem = """
Design an algorithm to find the longest palindromic substring in a string.

Requirements:
- Handle edge cases (empty string, single character, no palindromes)
- Optimize for time complexity
- Provide the implementation in Python
- Explain the approach and time/space complexity
"""

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "high"},
    input=[{"role": "user", "content": problem}]
)

print("Model response:\n")
print(response.output_text)
print(f"\n{'='*60}")
print(f"Reasoning tokens: {response.usage.output_tokens_details.reasoning_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")

Reasoning effort = high (longest palindromic substring)
------------------------------------------------------------
Model response:

### Approach (Manacher’s Algorithm — **O(n)** time)

A palindrome expands symmetrically around its center. A brute-force “expand around center” approach checks every center and expands, but it can take **O(n²)** in the worst case (e.g., `"aaaaaa..."`).

**Manacher’s algorithm** optimizes this by reusing information from previously found palindromes, achieving **O(n)** time.

#### Key idea
1. **Transform the string** to handle even/odd-length palindromes uniformly by inserting separators:
   - Original: `abba`
   - Transformed: `^#a#b#b#a#$`
   - `^` and `$` are sentinels to avoid bounds checks.
2. Maintain:
   - `C`: center of the rightmost palindrome found so far
   - `R`: right boundary of that palindrome
   - `P[i]`: radius of palindrome around position `i` in the transformed string
3. For each position `i`, use its mirror around `C` to initialize `P[

In [10]:
# Side-by-side comparison of reasoning efforts
print("Reasoning effort side-by-side comparison")
print("-" * 60)

task = """
You have a list of meeting times in 24-hour format as strings: 
["09:00-10:30", "10:00-11:00", "14:00-15:30", "15:00-16:00"]

Write a Python function that finds all overlapping meetings.
Return a list of tuples showing which meetings overlap.
"""

efforts = ["none", "low", "medium", "high"]
results = {}

for effort in efforts:
    response = client.responses.create(
        model="gpt-5.2",
        reasoning={"effort": effort},
        input=[{"role": "user", "content": task}]
    )

    results[effort] = {
        "output": response.output_text,
        "reasoning_tokens": response.usage.output_tokens_details.reasoning_tokens,
        "output_tokens": response.usage.output_tokens,
        "total_tokens": response.usage.total_tokens
    }

# Display comparison
for effort in efforts:
    print(f"\n{'='*70}")
    print(f"REASONING EFFORT: {effort.upper()}")
    print(f"{'='*70}")
    print(
        "Token usage: "
        f"Reasoning={results[effort]['reasoning_tokens']}, "
        f"Output={results[effort]['output_tokens']}, "
        f"Total={results[effort]['total_tokens']}"
    )
    print(f"\nOutput preview (first 300 chars):\n{results[effort]['output'][:300]}...")

Reasoning effort side-by-side comparison
------------------------------------------------------------

REASONING EFFORT: NONE
Token usage: Reasoning=0, Output=316, Total=395

Output preview (first 300 chars):
```python
from typing import List, Tuple

def find_overlapping_meetings(meetings: List[str]) -> List[Tuple[str, str]]:
    def to_minutes(t: str) -> int:
        h, m = map(int, t.split(":"))
        return 60 * h + m

    def parse(slot: str) -> tuple[int, int]:
        start, end = slot.split("-")...

REASONING EFFORT: LOW
Token usage: Reasoning=0, Output=307, Total=386

Output preview (first 300 chars):
```python
from typing import List, Tuple

def find_overlapping_meetings(meetings: List[str]) -> List[Tuple[str, str]]:
    def to_minutes(t: str) -> int:
        h, m = map(int, t.split(":"))
        return h * 60 + m

    parsed = []
    for s in meetings:
        start_s, end_s = s.split("-")
    ...

REASONING EFFORT: MEDIUM
Token usage: Reasoning=120, Output=383, Total=462



<a id='verbosity'></a>
## 4. Verbosity Control

GPT-5.2 introduces a **verbosity** parameter to control output length. This is **independent from reasoning depth** - you can have high reasoning with concise output, or low reasoning with verbose output.

| Verbosity | Description |
|-----------|-------------|
| **low** | Concise, bullet-point style answers |
| **medium** | Balanced explanations (default) |
| **high** | Detailed, comprehensive responses |

Here we keep reasoning effort fixed and vary verbosity to show length control.
Notice how the answer length changes while the core meaning stays similar.

In [8]:
question = "What is dependency injection and why is it useful?"

print("Verbosity comparison (same question, different lengths)")
print("-" * 60)

# Try different verbosity levels
for verbosity in ["low", "medium", "high"]:
    response = client.responses.create(
        model="gpt-5.2",
        reasoning={"effort": "medium"},
        text={"verbosity": verbosity},
        input=[{"role": "user", "content": question}]
    )

    print(f"\n{'='*60}")
    print(f"VERBOSITY: {verbosity.upper()}")
    print(f"{'='*60}")
    output_text = response.output_text
    print(output_text[:500] + "..." if len(output_text) > 500 else output_text)
    print(f"\nOutput tokens: {response.usage.output_tokens}")

Verbosity comparison (same question, different lengths)
------------------------------------------------------------

VERBOSITY: LOW
Dependency injection (DI) is a design pattern where an object *receives* the other objects (“dependencies”) it needs from the outside, instead of creating them itself (e.g., via `new` inside the class). Those dependencies are typically provided through a constructor, a setter, or a framework/container.

### What it looks like (idea)
Without DI (class constructs its dependency):
```java
class OrderService {
  private final PaymentGateway gateway = new StripeGateway(); // hard-coded
}
```

With DI...

Output tokens: 300

VERBOSITY: MEDIUM
Dependency Injection (DI) is a design pattern for building software where an object (a “dependent”) does **not create** the other objects it needs (its “dependencies”). Instead, those dependencies are **provided (“injected”) from the outside**, usually by a framework, a factory, or a caller.

### What it looks like
**Witho

<a id='tools'></a>
## 5. Tool Use with Reasoning

GPT-5.2 can use tools (function calling) while reasoning. The model will think about which tools to use and how to use them.

These examples show tool calls and a web search tool.
If the model decides a tool call is needed, you will see tool call metadata.

In [13]:
def send_email(to, subject, body):
    print(f"Sending email to {to} with subject {subject} and body {body}")

response = client.responses.create(
    model="gpt-5.2",
    instructions="""You are a helpful email assistant with access 
    to an email tool: send_email(to, subject, body), if the user asks for an email to be sent
    the output should be just the tool call: likeL: 'send_email(to, subject, body)'.
    """,
    input=[{"role": "user", "content": "Send an email to Lucas under lucasinstructor@gmail.com with subject 'hi' and body: 'lucas you are great!'"}]
)

response.output_text

'send_email("lucasinstructor@gmail.com", "hi", "lucas you are great!")'

In [14]:
eval(response.output_text)

Sending email to lucasinstructor@gmail.com with subject hi and body lucas you are great!


In [16]:
# Define tools for function calling
print("Tool calling example (model may request tool calls)")
print("-" * 60)

tools = [
    {
            "type": "function",
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g., 'San Francisco'"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    }
                },
                "required": ["location"]
            }
        }
]

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "medium"},
    tools=tools,
    input=[
        {
            "role": "user",
            "content": "What's the weather like in Tokyo and New York today?"
        }
    ]
)

print("Response text:", response.output_text if response.output_text else "(tool calls requested)")
if hasattr(response, 'output') and response.output:
    for item in response.output:
        if hasattr(item, 'type') and item.type == 'function_call':
            print(f"Tool call requested: {item.name}({item.arguments})")

Tool calling example (model may request tool calls)
------------------------------------------------------------
Response text: (tool calls requested)
Tool call requested: get_weather({"location":"Tokyo","unit":"celsius"})
Tool call requested: get_weather({"location":"New York","unit":"celsius"})


In [17]:
# Built-in web search tool
print("Web search tool example (live data)")
print("-" * 60)

response = client.responses.create(
    model="gpt-5.2",
    tools=[{"type": "web_search_preview"}],
    input=[{"role": "user", "content": "What were the major tech announcements this week?Output in markdown format."}]
)

print("Search-based response:\n")
print(response.output_text)

Web search tool example (live data)
------------------------------------------------------------
Search-based response:

## Major tech announcements this week (US week of **Feb 2–Feb 5, 2026**)

### OpenAI: **“Frontier”** enterprise platform for AI agents
- OpenAI announced **Frontier**, a platform for enterprises to **build, deploy, and manage AI agents** (“AI co‑workers”) that can work with files, analyze data, and execute code, with integrations into common business systems. ([axios.com](https://www.axios.com/2026/02/05/openai-platform-ai-agents?utm_source=openai))

### Google Search/Discover: **February 2026 Discover “core update”**
- Google rolled out a **broad update** to systems that rank/surface content in **Google Discover**, starting with **English-language users in the US**, with a rollout expected to take **up to two weeks**. ([searchengineland.com](https://searchengineland.com/google-releases-discover-core-update-february-2026-468308?utm_source=openai))

### Nintendo: **Sw

In [19]:
from IPython.display import Markdown

Markdown(response.output_text)

## Major tech announcements this week (US week of **Feb 2–Feb 5, 2026**)

### OpenAI: **“Frontier”** enterprise platform for AI agents
- OpenAI announced **Frontier**, a platform for enterprises to **build, deploy, and manage AI agents** (“AI co‑workers”) that can work with files, analyze data, and execute code, with integrations into common business systems. ([axios.com](https://www.axios.com/2026/02/05/openai-platform-ai-agents?utm_source=openai))

### Google Search/Discover: **February 2026 Discover “core update”**
- Google rolled out a **broad update** to systems that rank/surface content in **Google Discover**, starting with **English-language users in the US**, with a rollout expected to take **up to two weeks**. ([searchengineland.com](https://searchengineland.com/google-releases-discover-core-update-february-2026-468308?utm_source=openai))

### Nintendo: **Switch 2 game lineup revealed (Nintendo Direct)**
- Nintendo unveiled a **Switch 2** lineup featuring major third‑party releases (e.g., *Fallout 4: Anniversary Edition*, *FF7 Rebirth*, *Resident Evil* titles) with several **dated releases** spanning Feb–June 2026 and beyond. ([theverge.com](https://www.theverge.com/games/874282/nintendo-direct-february-2026-fallout-ff7?utm_source=openai))

### Microsoft/Xbox: **Game Pass February 2026 “Wave 1” lineup**
- Microsoft announced the first **February 2026 Wave 1** additions to **Xbox Game Pass**, with titles arriving starting **Feb 3** and continuing through mid‑month (e.g., *Madden NFL 26* on **Feb 5**, *High on Life 2* on **Feb 13**). ([news.xbox.com](https://news.xbox.com/en-us/2026/02/03/xbox-game-pass-february-2026-wave-1/?utm_source=openai))

### Samsung (mobile): **Galaxy F70e 5G launch date + key specs (India)**
- Samsung announced **Galaxy F70e 5G** will debut **Feb 9, 2026**, highlighting features like a **50MP camera** and **120Hz** display. ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/mobiles-tabs/samsung-galaxy-f70e-5g-to-debut-on-february-9-50mp-camera-120hz-refresh-rate-and-more/articleshow/127862062.cms?utm_source=openai))

If you tell me what you consider “major” (AI models, consumer gadgets, enterprise/cloud, gaming, startups), I can tighten this to just the top items in that category.

<a id='structured'></a>
## 6. Structured Outputs

Use Pydantic models to get structured, typed responses from GPT-5.2.

We define a Pydantic schema and ask the model to return JSON that fits it.
You should see a parsed, typed result with labeled fields.

In [21]:
from pydantic import BaseModel, Field
from typing import List
import json

print("Structured output example (JSON schema)")
print("-" * 60)

class CodeReview(BaseModel):
    """Structured code review output"""
    issues: List[str] = Field(description="List of identified issues")
    suggestions: List[str] = Field(description="Improvement suggestions")
    severity: str = Field(description="Overall severity: low, medium, high")
    score: int = Field(description="Code quality score from 1-10")

code_to_review = """
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)
"""

response = client.responses.parse(
    model="gpt-5.2",
    reasoning={"effort": "medium"},
    input=[
        {
            "role": "developer",
            "content": "Review code for bugs, edge cases, and improvements."
        },
        {
            "role": "user",
            "content": f"Review this code:\n```python\n{code_to_review}\n```"
        }
    ],
    text_format=CodeReview
)

review = response.output_parsed
print(f"Score: {review.score}/10")
print(f"Severity: {review.severity}")
print("\nIssues:")
for issue in review.issues:
    print(f"  - {issue}")
print("\nSuggestions:")
for suggestion in review.suggestions:
    print(f"  - {suggestion}")

Structured output example (JSON schema)
------------------------------------------------------------
Score: 6/10
Severity: medium

Issues:
  - Division by zero when `numbers` is empty (`len(numbers) == 0`).
  - Assumes `numbers` supports `len()`; generators/iterators will raise `TypeError`.
  - No input validation: non-numeric items (e.g., strings, `None`) will cause `TypeError` during addition or division.
  - Potentially ambiguous behavior for non-standard numeric types (e.g., `Decimal`, `Fraction`) when mixing with `int`/`float`, depending on inputs.

Suggestions:
  - Handle the empty case explicitly (e.g., raise `ValueError` or return `0.0` depending on desired semantics).
  - If you want to support any iterable (including generators), either convert to a list/tuple first or compute count manually while iterating.
  - Use Python’s built-in `sum()` for clarity and potential performance improvements: `return sum(numbers) / len(numbers)` (still needs empty handling).
  - Consider addi

<a id='best-practices'></a>
## 7. Best Practices

### Write Briefs, Not Prompts

Reasoning models work best with comprehensive context, not chat-style prompts.

### Focus on WHAT, Not HOW

Let the model decide how to approach the problem. Don't micromanage the reasoning process.

### Don't Ask for Chain-of-Thought

Reasoning models already think internally - asking them to "think step-by-step" can actually degrade performance.

Two short examples: prompt brief structure and a quick effort-level guide.
Use these as templates for your own tasks.

In [ ]:
# BAD: Chat-style prompt
print("Prompting best practices: bad vs good brief")
print("-" * 60)

bad_prompt = "Can you help me optimize this function?"

# GOOD: Brief-style prompt with full context
good_prompt = """
CONTEXT:
I'm building a high-performance mathematics library for a financial trading system.
The library needs to handle real-time calculations with microsecond precision.

CURRENT IMPLEMENTATION:
def calculate_fibonacci(n):
    if n <= 1:
        return n
    return calculate_fibonacci(n-1) + calculate_fibonacci(n-2)

PROBLEMS:
- Exponential time complexity O(2^n)
- Stack overflow for n > 1000
- Called millions of times per second in production

REQUIREMENTS:
1. Optimize for speed (target: < 1 microsecond for n < 100)
2. Handle large values (n up to 10,000)
3. Thread-safe implementation

DELIVERABLE:
Provide a production-ready Python implementation with time complexity analysis.
"""

print("BAD (lazy prompt):")
print(bad_prompt)
print("\n" + "="*60 + "\n")
print("GOOD (comprehensive brief):")
print(good_prompt)

In [ ]:
# Choosing the Right Effort Level
print("Effort level selection guide")
print("-" * 60)

effort_guide = {
    "none": ["Simple classification", "Data extraction", "Formatting"],
    "low": ["Basic Q&A", "Simple code fixes", "Summarization"],
    "medium": ["Code generation", "Data analysis", "Most general tasks"],
    "high": ["Algorithm design", "Complex debugging", "Architecture planning"],
    "xhigh": ["Research problems", "Novel algorithm design", "Critical accuracy tasks"]
}

print("Effort Level Selection Guide:")
print("="*60)
for effort, use_cases in effort_guide.items():
    print(f"\n{effort.upper()}:")
    for case in use_cases:
        print(f"  - {case}")

<a id='examples'></a>
## 8. Practical Examples

Each example below is a full mini-brief. Read the brief first, then run the cell
and compare the model output to the desired deliverable.

In [ ]:
# Example: Code Review with Medium Reasoning
print("Practical example: code review (medium effort)")
print("-" * 60)

buggy_code = """
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)

def process_user_data(data):
    result = {}
    for item in data:
        result[item['id']] = item['name'].upper()
    return result
"""

prompt = f"""
Review this Python code for potential bugs, edge cases, and issues.
For each issue found:
1. Describe the problem
2. Show what input would cause it to fail
3. Provide a fix

Code:
```python
{buggy_code}
```
"""

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}]
)

print("Code review output:\n")
print(response.output_text)

In [ ]:
# Example: Business Analysis with High Reasoning
print("Practical example: business analysis (high effort)")
print("-" * 60)

analysis_brief = """
COMPANY CONTEXT:
TechStartup Inc. - B2B SaaS platform for inventory management
- Current MRR: $125,000
- Customer Count: 187
- Monthly Churn Rate: 5.2%
- Customer Acquisition Cost (CAC): $3,200
- Customer Lifetime Value (CLV): $18,500

CHALLENGE:
High churn in SMB segment (8.5% monthly) vs Enterprise (2.1% monthly)

DELIVERABLE:
Provide 3 prioritized recommendations to reduce SMB churn, with expected impact.
"""

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "high"},
    text={"verbosity": "medium"},
    input=[{"role": "user", "content": analysis_brief}]
)

print("Analysis output:\n")
print(response.output_text)

In [ ]:
# Example: Algorithm Design with High Reasoning
print("Practical example: algorithm design (high effort)")
print("-" * 60)

algorithm_brief = """
PROBLEM:
Design a rate limiter for an API gateway.

REQUIREMENTS:
- Support 10,000 requests/second per user
- Distributed across 50 servers
- Must handle clock skew between servers
- Support burst traffic (allow 2x normal rate for 5 seconds)

DELIVERABLE:
1. Algorithm design with pseudocode
2. Data structure choices with justification
3. How to handle the distributed nature
4. Trade-offs and limitations
"""

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "high"},
    input=[{"role": "user", "content": algorithm_brief}]
)

print("Architecture output:\n")
print(response.output_text)

<a id='summary'></a>
## 9. Summary

### Key Takeaways

1. **Use the Responses API** - `client.responses.create()` is designed for reasoning models

2. **Choose the right effort level**:
   - `none`/`low`: Simple tasks, fast responses
   - `medium`: Default for most tasks (best balance)
   - `high`/`xhigh`: Complex problems requiring deep reasoning

3. **Control verbosity separately** - Reasoning depth and output length are independent

4. **Write briefs, not prompts** - Provide comprehensive context for best results

5. **Don't ask for chain-of-thought** - The model reasons internally automatically

6. **Use developer role** - Set system-level context with `{"role": "developer"}`

### When to Use Reasoning Models

**Great Use Cases:**
- Complex code generation and debugging
- Multi-step problem solving
- Data analysis with multiple considerations
- Algorithm design
- Strategic planning

**Probably Overkill:**
- Simple text generation
- Basic Q&A
- Translation
- Simple formatting tasks

### Resources

- [OpenAI Responses API Documentation](https://platform.openai.com/docs/api-reference/responses)
- [GPT-5.2 Model Card](https://platform.openai.com/docs/models#gpt-5)
- [Reasoning Best Practices](https://platform.openai.com/docs/guides/reasoning)